In [1]:
import os 
# change wd
os.chdir('..')
!pwd

/Users/cshatto/Projects/fire-recovery


In [2]:
import geopandas as gpd


In [6]:
gdf = gpd.read_file('data/output/burn_areas.geojson')

In [7]:
gdf

,geometry
0,"POLYGON ((-8.063 41.482, -8.063 41.481, -8.062..."
1,"POLYGON ((-8.096 41.479, -8.096 41.479, -8.095..."
2,"POLYGON ((-8.225 41.474, -8.225 41.473, -8.225..."
3,"POLYGON ((-8.198 41.474, -8.198 41.473, -8.197..."
4,"POLYGON ((-8.202 41.472, -8.202 41.472, -8.201..."
...,...
827,"POLYGON ((-7.894 41.068, -7.894 41.067, -7.893..."
828,"POLYGON ((-8.006 41.072, -8.006 41.071, -8.008..."
829,"POLYGON ((-7.974 41.097, -7.974 41.096, -7.977..."
830,"POLYGON ((-7.848 41.13, -7.848 41.13, -7.847 4..."


In [ ]:
from dotenv import load_dotenv
import os
import requests

def get_cdse_token():
    """Generate and return an access token for Copernicus Data Space Ecosystem."""
    load_dotenv('.env.local')
    copernicus_user = os.getenv('COPERNICUS_USER')
    copernicus_password = os.getenv('COPERNICUS_PASS')
    
    if not copernicus_user or not copernicus_password:
        raise ValueError("Missing COPERNICUS_USER or COPERNICUS_PASS in .env.local")
    
    data = {
        "client_id": "cdse-public",
        "username": copernicus_user,
        "password": copernicus_password,
        "grant_type": "password"
    }
    
    try:
        response = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data
        )
        response.raise_for_status()
        return response.json()["access_token"]
    except requests.exceptions.RequestException as e:
        raise Exception(f"Failed to obtain CDSE access token: {str(e)}")

In [9]:
# Import libraries
import requests
import pandas as pd
from shapely.geometry import box

# Define AOI (Lisbon, Portugal, in WGS84)
minx, miny, maxx, maxy = -9.3, 38.6, -9.0, 38.9
aoi = box(minx, miny, maxx, maxy).wkt

# Query parameters
start_date = "2023-01-01T00:00:00.000Z"
end_date = "2023-12-31T23:59:59.999Z"
collection = "SENTINEL-2"
product_type = "S2MSI2A"
cloud_cover = "[0 TO 20]"

# OData query
query = (
    f"Products?$filter=Collection/Name eq '{collection}' "
    f"and Attributes/OData.CSC.DoubleAttribute/any(d:d/Name eq 'cloudCover' and d/Value {cloud_cover}) "
    f"and ContentDate/Start gt {start_date} and ContentDate/Start lt {end_date} "
    f"and OData.CSC.Intersects(area=geography'{aoi}')"
)

# Query API
base_url = "https://catalogue.dataspace.copernicus.eu/odata/v1/"
try:
    token = get_cdse_token()
    print("Authentication successful!")
except Exception as e:
    print(f"Authentication failed: {e}")
    raise

headers = {"Authorization": f"Bearer {token}"}
response = requests.get(f"{base_url}{query}", headers=headers)
response.raise_for_status()

# Display results
products = response.json()["value"]
if not products:
    print("No products found.")
else:
    df = pd.DataFrame([
        {
            "Name": p["Name"],
            "Date": p["ContentDate"]["Start"],
            "CloudCover": next((attr["Value"] for attr in p["Attributes"] if attr["Name"] == "cloudCover"), None),
            "ProductID": p["Id"]
        }
        for p in products
    ])
    print(f"Found {len(products)} products:")
    display(df)

Authentication failed: Missing COPERNICUS_USER or COPERNICUS_PASSWORD in .env.local


ValueError: Missing COPERNICUS_USER or COPERNICUS_PASSWORD in .env.local